# softmax回归的简洁实现
我们在3.3节（线性回归的简洁实现）中已经了解了使用Pytorch实现模型的便利。
下面，让我们再次使用Pytorch来实现一个softmax回归模型。
首先导入所需的包或模块。

In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import d2lzh_torch as d2l

## 获取和读取数据
我们仍然使用Fashion-MNIST数据集和上一节中设置的批量大小。

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
train_iter,test_iter

(<torch.utils.data.dataloader.DataLoader at 0x12184eba8>,
 <torch.utils.data.dataloader.DataLoader at 0x12184ea58>)

## 定义和初始化模型
在3.4节（softmax回归）中提到，softmax回归的输出层是一个全连接层，所以我们用一个线性模块就可以了。
因为前面我们数据返回的每个batch样本x的形状为(batch_size, 1, 28, 28), 所以我们要先用view()将x的形状转换成(batch_size, 784)才送入全连接层。

In [3]:
num_inputs = 784
num_outputs = 10

class LinearNet(nn.Module):
    def __init__(self,num_inputs, num_outputs):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
        
    def forward(self,x):
        y = self.linear(x.view(x.shape[0], -1))
        return y
    
net = LinearNet(num_inputs, num_outputs)
        

我们将对x的形状转换的这个功能自定义一个FlattenLayer并记录在d2lzh_pytorch中方便后面使用。

In [4]:
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer,self).__init__()
    
    def forward(self,x):
        return x.view(x.shape[0], -1)
        

这样我们就可以更方便地定义我们的模型：

In [5]:
from collections import OrderedDict

net = nn.Sequential(OrderedDict([
    ('flatten',FlattenLayer()),
    ('linear',nn.Linear(num_inputs, num_outputs))
]))

然后，我们使用均值为0、标准差为0.01的正态分布随机初始化模型的权重参数。

In [6]:
init.normal_(net.linear.weight,mean = 0,std = 0.01)
init.constant_(net.linear.bias,val = 0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

## softmax和交叉熵损失函数
如果做了上一节的练习，那么你可能意识到了分开定义softmax运算和交叉熵损失函数可能会造成数值不稳定。
因此，PyTorch提供了一个包括softmax运算和交叉熵损失计算的函数。它的数值稳定性更好。

In [7]:
loss = nn.CrossEntropyLoss()

## 定义优化算法
我们使用学习率为0.1的小批量随机梯度下降作为优化算法。

In [8]:
optimizer = torch.optim.SGD(net.parameters(),lr = 0.1)

## 训练模型
接下来，我们使用上一节中定义的训练函数来训练模型。

In [9]:
num_epochs = 5
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0031, train acc 0.749, test acc 0.769
epoch 2, loss 0.0022, train acc 0.812, test acc 0.803
epoch 3, loss 0.0021, train acc 0.825, test acc 0.817
epoch 4, loss 0.0020, train acc 0.832, test acc 0.823
epoch 5, loss 0.0019, train acc 0.838, test acc 0.825


softmax运算和交叉熵损失函数可能会造成数值不稳定问题

参考[softmax和交叉熵](https://blog.csdn.net/behamcheung/article/details/71911133#softmax%E7%9A%84%E8%AE%A1%E7%AE%97%E4%B8%8E%E6%95%B0%E5%80%BC%E7%A8%B3%E5%AE%9A%E6%80%A7)